In [2]:
%load_ext autoreload
%autoreload 2
%cd ~/big_vision
import io
import os
import jax
import json
import torch
import importlib
import numpy as np
import ml_collections
import jax.numpy as jnp
import tensorflow as tf

import open_clip
import big_vision.utils as u
import big_vision.models.vit as model_mod
from big_vision.models.vit import scan_to_pyloop

import nest_asyncio
nest_asyncio.apply()

/home/gavin/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/gavin/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/gavin/big_vision


2024-10-29 23:08:28.672465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730243308.694856   19060 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730243308.701758   19060 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/gavin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Hyperparameters
save_bv_vit = True
backbone = 'cappa_s9b_bs32k'

# setup
backbone_dict = {
    'clip': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/clip_bs16384_warm10k_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_12lyr_07-23_1510',
    'clip_map': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/clip_autoregressive_bs16384_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_12lyr_06-24_2019',
    'clip_s9b': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/clip_autoregressive_s9b_bs16384_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_12lyr_08-04_0839',
    'clip_s9b_bs32k': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/clip_s9b_bs32k_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_08-09_0655',
    'siglip': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_parallel_bs16384_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_12lyr_06-24_2019',
    'siglip_v4-32': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_replication_pod_04-11_2247',
    'siglip_s9b_bs32k': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_s9b_bs32k_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_08-04_0839',
    'cappa': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/cappa_bs16384_s3B_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_06-27_2108',
    # 'cappa_decoder-qknorm-T_warm0.02': 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_warm0.02_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-15_2102',
    'cappa_s9b': 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_s9B_warm0.02_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_06-27_2108',
    'cappa_s9b_bs32k': 'gs://us-central2-storage/tensorflow_datasets/cappa_bs32768_s9B_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_08-07_2217',
    'coca_6lyr': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_replication_bs16384_warm0.03_1.0co-2.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_06-30_1841',
    'coca_unified': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_replication_s3b_bs16384_warm0.03_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_08-19_0355',
    'coca_1.0co_1.0ca_6lyr_qknorm-T_warm0.02': 'gs://us-central2-storage/tensorflow_datasets/ckpts/coca_replication_bs16384_warm0.02_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-10_2225',
    'coca': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_replication_bs32768_warm0.03_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_08-12_2313',
    'coca_6lyr_1.0co_1.0ca_s9b_bs32k': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_replication_s9b_bs32768_warm0.03_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_08-14_1614',
    'gemma2b-half-0.1_b16-F_contrastive': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-half-0.1_so400m-F_contrastive_bs16384_s3b_wd1e-4_08-21_1935',
    'gemma2b-contrastive_9+9_llm_gap_0.01': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-gap_b16-F_contrastive_bs16k_s3b_lr1e-3_wd1e-4_bf16_09-01_0446',
    'gemma2b-contrastive_ffn_llm_1.0': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-adapter_ffn-1.0-drop0.0-vocab256128-projF_b16-beitF-qknormF_contrastive_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-08_1756',
    'gemma2b-full-0.1_b16-F_generative': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-full-0.1_so400m-F_generative_bs16384_s3b_wd1e-4_08-25_0445',
    'gemma2b-12+6-0.01_b16_generative': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen12-0.01-drop0.0-none_b16-beitF_contrastive_bs16k_s3b_lr1e-3_wd1e-4_bf16_09-08_0448',
    'gemma2b_9+9_llm_0.02_b16_qknormT_generative': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.02-drop0.0-none_b16-beitF-qknormT_generative_bs16k_s3b_lr1e-3_wd1e-4_fp32_10-02_1429',
    'coca_6lyr_s9b_bs32k': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_unified_s9b_bs32768_warm0.03_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_08-20_0246',
    'coca_unified_1.0co_0.0ca': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_unified_s3b_bs16384_warm0.03_1.0co-0.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_09-05_0535',
    'coca_unified_0.0co_1.0ca': 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_unified_s3b_bs16384_warm0.03_0.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_09-05_0603',
    'clip_FT_50M_recapF': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_s3b_bs16k_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapFalse_50M_10-02_2028',
    'clip_FT_50M_recapT': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_s3b_bs16k_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapTrue_50M_10-02_2028',
    'clip+gemma_FT_recapF': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_s3b_bs16k+gemma2b_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapFalse_10-03_0440',
    'clip+gemma_FT_recapT': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_s3b_bs16k+gemma2b_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapTrue_10-03_0440',
    'gemma-supervised_FT_recapF': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/gemma-supervised_s3b_bs16k_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapFalse_10-02_1727',
    'gemma-supervised_FT_recapT': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/gemma-supervised_s3b_bs16k_lr1e-5_wd0.0_bs16k_epoch5_warm0.03_recapTrue_10-02_1437',
    'gemma-supervised_PT_recapF': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-drop0.0-vocab256128-gap-projT_b16-beitF-qknormF_contrastive_datacomp-recapF-50M-epoch10_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-09_2043',
    'gemma-supervised_PT_recapT': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-drop0.0-vocab256128-gap-projT_b16-beitF-qknormF_contrastive_datacomp-recapT-50M-epoch10_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-09_2039',
    'gemma-supervised_PT_org0.5': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-drop0.0-vocab256128-gap-projT_b16-beitF-qknormF_contrastive_datacomp-org_ratio0.5-50M-epoch10_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-11_0801',
    'gemma-supervised-generative_PT_org_10M': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-drop0.0-vocab256128-gap-projT-txtlen128_b16-beitF-qknormF_generative_datacomp-org_ratio1.0-10M-epoch10_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-17_1650',
    'clip_PT_recapF': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_PT_lr1e-3_wd1e-4_bs16k_epoch10_warm0.03_recapFalse_50M_10-10_0458',
    'clip_PT_recapT': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_PT_lr1e-3_wd1e-4_bs16k_epoch10_warm0.03_recapTrue_50M_10-10_1226',
    'clip_PT_org0.5': 'gs://us-central2-storage/tensorflow_datasets/post-training-datacomp-recap/clip_PT_lr1e-3_wd1e-4_bs16k_epoch10_warm0.03_org-ratio0.5_50M_10-11_1516',
    'llava_gemma_PT_cambrian10M-10epoch': 'gs://us-central2-storage/tensorflow_datasets/mllm_ckpts/paligemma/gemma2b-partial_frozen99-0.01-drop0.0-vocab256128-gap-projT-txtlen128_b16-beitF-qknormF_generative_datacomp-org_ratio1.0-10M-epoch10_dpr0.0_bs16k_s3b_lr1e-3_wd1e-4_bf16_10-29_1338',
}
backbone_path = backbone_dict[backbone]
gcs_np_save_path = f'{backbone_path}/bv_{backbone}.npz'
local_np_save_path = f'./ckpt_npz/bv_{backbone}.npz'
if not os.path.exists('./ckpt_npz'): os.makedirs('./ckpt_npz')
config_path = f'{backbone_path}/config.json'
metrics_path = f'{backbone_path}/big_vision_metrics.txt'

config = ml_collections.ConfigDict(json.load(tf.io.gfile.GFile(config_path, 'r')))
bv_metrics = json.loads(tf.io.gfile.GFile(metrics_path, 'r').read().split('\n')[-2])
ckpt_path = f'{backbone_path}/checkpoint.bv-{config.total_steps:09d}'

# big_vision ViT

In [4]:
model_cfg = config.model
img_key = 'img' if 'image' in model_cfg or 'img' in model_cfg else 'encoder'

# load model
model_mod = importlib.import_module(f"big_vision.models.{config.model_name}")
bv_model = model_mod.Model(**model_cfg)

# load ckpt weights
rng = jax.random.PRNGKey(42)
dummy_img = jnp.zeros([2, 224, 224, 3], jnp.float32)
dummy_txt = jnp.zeros([2, 64], jnp.int32)
dummy_mask_ar = jnp.zeros([2, 64], jnp.bool_) if 'llm' in model_cfg else None
if dummy_mask_ar is not None:
    init_params = jax.jit(bv_model.init, backend="cpu")(rng, dummy_img, dummy_txt,dummy_mask_ar)['params']
else:
    init_params = jax.jit(bv_model.init, backend="cpu")(rng, dummy_img, dummy_txt)['params']

# img_load_kw = {'dont_load': ('.*_ln/scale','head/kernel', 'head/bias')}
params = model_mod.load(init_params, ckpt_path, model_cfg) #, img_load_kw)
img_params = params[img_key]
# jax.tree_map(jnp.shape, img_params)
jax.tree_map(jnp.dtype, img_params)

I1029 23:08:49.851347   20417 google_auth_provider.cc:181] Running on GCE, using service account 373177222751-compute@developer.gserviceaccount.com
/tmp/ipykernel_19060/1131154012.py:22: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(jnp.dtype, img_params)


{'Transformer': {'encoder_norm': {'bias': dtype('float32'),
   'scale': dtype('float32')},
  'encoderblock': {'LayerNorm_0': {'bias': dtype('float32'),
    'scale': dtype('float32')},
   'LayerNorm_1': {'bias': dtype('float32'), 'scale': dtype('float32')},
   'MlpBlock_0': {'Dense_0': {'bias': dtype('float32'),
     'kernel': dtype('float32')},
    'Dense_1': {'bias': dtype('float32'), 'kernel': dtype('float32')}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': dtype('float32'),
     'kernel': dtype('float32')},
    'out': {'bias': dtype('float32'), 'kernel': dtype('float32')},
    'query': {'bias': dtype('float32'), 'kernel': dtype('float32')},
    'value': {'bias': dtype('float32'), 'kernel': dtype('float32')}}}},
 'embedding': {'bias': dtype('float32'), 'kernel': dtype('float32')},
 'pos_embedding': dtype(bfloat16)}

In [5]:
if save_bv_vit:
    # save big_vision backbone vit as npz file
    
    ckpt = {'params': {'img': scan_to_pyloop(img_params) if 'encoderblock' in img_params['Transformer'].keys() else img_params}}
    names_and_vals, _ = u.tree_flatten_with_names(ckpt)
    names_and_vals = [(n, v.astype(np.float32)) for n, v in names_and_vals] # convert bf16 pos_emb to fp32 to avoid void np.dtype error
    io_buffer = io.BytesIO()
    np.savez(io_buffer, **{k: v for k, v in names_and_vals})

    # save to gcs
    with tf.io.gfile.GFile(gcs_np_save_path, "wb") as f: f.write(io_buffer.getvalue())
    # save to local
    with open(local_np_save_path, "wb") as f: f.write(io_buffer.getvalue())


big_vision_vit = u.npload(gcs_np_save_path)
# inspect keys, shapes, and dtypes
for key in big_vision_vit.keys():
    print(key, big_vision_vit[key].shape, big_vision_vit[key].dtype)

params/img/Transformer/encoder_norm/bias (768,) float32
params/img/Transformer/encoder_norm/scale (768,) float32
params/img/Transformer/encoderblock_0/LayerNorm_0/bias (768,) float32
params/img/Transformer/encoderblock_0/LayerNorm_0/scale (768,) float32
params/img/Transformer/encoderblock_0/LayerNorm_1/bias (768,) float32
params/img/Transformer/encoderblock_0/LayerNorm_1/scale (768,) float32
params/img/Transformer/encoderblock_0/MlpBlock_0/Dense_0/bias (3072,) float32
params/img/Transformer/encoderblock_0/MlpBlock_0/Dense_0/kernel (768, 3072) float32
params/img/Transformer/encoderblock_0/MlpBlock_0/Dense_1/bias (768,) float32
params/img/Transformer/encoderblock_0/MlpBlock_0/Dense_1/kernel (3072, 768) float32
params/img/Transformer/encoderblock_0/MultiHeadDotProductAttention_0/key/bias (12, 64) float32
params/img/Transformer/encoderblock_0/MultiHeadDotProductAttention_0/key/kernel (768, 12, 64) float32
params/img/Transformer/encoderblock_0/MultiHeadDotProductAttention_0/out/bias (768,) 

# open_clip

In [6]:
@torch.no_grad()
def load_any_bv_vit(model, checkpoint_path, strict=True):
    print(f"HELLO! I changed the load_checkpoint function!!!!")
    from timm.layers import resample_patch_embed, resample_abs_pos_embed

    def _n2p(w, t=True):
        if w.ndim == 4 and w.shape[0] == w.shape[1] == w.shape[2] == 1:
            w = w.flatten()
        if t:
            if w.ndim == 4:
                w = w.transpose([3, 2, 0, 1])
            elif w.ndim == 3:
                w = w.transpose([2, 0, 1])
            elif w.ndim == 2:
                w = w.transpose([1, 0])

        # if w.dtype == np.dtype('|V2'):
        #     w_float16 = w.view(np.float16)
        #     w = w_float16.astype(np.float32)

        return torch.from_numpy(w)

    w = np.load(checkpoint_path)
    interpolation = 'bilinear'
    antialias = False
    
    module = model.visual.trunk
    prefix = 'params/img/'
    embed_conv_w = _n2p(w[f'{prefix}embedding/kernel'])
    if embed_conv_w.shape[-2:] != module.patch_embed.proj.weight.shape[-2:]:
        embed_conv_w = resample_patch_embed(
            embed_conv_w,
            module.patch_embed.proj.weight.shape[-2:],
            interpolation=interpolation,
            antialias=antialias,
            verbose=True,
        )
    module.patch_embed.proj.weight.copy_(embed_conv_w)
    module.patch_embed.proj.bias.copy_(_n2p(w[f'{prefix}embedding/bias']))

    if module.cls_token is not None:
        module.cls_token.copy_(_n2p(w[f'{prefix}cls'], t=False))

    pos_embed_w = _n2p(w[f'{prefix}pos_embedding'], t=False)
    if pos_embed_w.shape != module.pos_embed.shape:
        assert False, f'{pos_embed_w.shape}, {module.pos_embed.shape}'
        num_prefix_tokens = 0 if getattr(module, 'no_embed_class', False) else getattr(module, 'num_prefix_tokens', 1)
        pos_embed_w = resample_abs_pos_embed(  # resize pos embedding when different size from pretrained weights
            pos_embed_w,
            new_size=module.patch_embed.grid_size,
            num_prefix_tokens=num_prefix_tokens,
            interpolation=interpolation,
            antialias=antialias,
            verbose=True,
        )
    module.pos_embed.copy_(pos_embed_w)

    mha_sub, b_sub, ln1_sub = (0, 0, 1)
    for i, block in enumerate(module.blocks.children()):
        block_prefix = f'{prefix}Transformer/encoderblock_{i}/'
        mha_prefix = block_prefix + f'MultiHeadDotProductAttention_{mha_sub}/'
        block.norm1.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/scale']))
        block.norm1.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/bias']))
        block.attn.qkv.weight.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/kernel'], t=False).flatten(1).T for n in ('query', 'key', 'value')]))
        block.attn.qkv.bias.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/bias'], t=False).reshape(-1) for n in ('query', 'key', 'value')]))
        block.attn.proj.weight.copy_(_n2p(w[f'{mha_prefix}out/kernel']).flatten(1))
        block.attn.proj.bias.copy_(_n2p(w[f'{mha_prefix}out/bias']))
        for r in range(2):
            getattr(block.mlp, f'fc{r + 1}').weight.copy_(_n2p(w[f'{block_prefix}MlpBlock_{b_sub}/Dense_{r}/kernel']))
            getattr(block.mlp, f'fc{r + 1}').bias.copy_(_n2p(w[f'{block_prefix}MlpBlock_{b_sub}/Dense_{r}/bias']))
        block.norm2.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_{ln1_sub}/scale']))
        block.norm2.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_{ln1_sub}/bias']))

    module_norm = module.norm if isinstance(module.norm,torch.nn.LayerNorm) else module.fc_norm
    module_norm.weight.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/scale']))
    module_norm.bias.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/bias']))

    if module.attn_pool is not None:
        block_prefix = f'{prefix}MAPHead_0/'
        mha_prefix = block_prefix + f'MultiHeadDotProductAttention_0/'
        module.attn_pool.latent.copy_(_n2p(w[f'{block_prefix}probe'], t=False))
        module.attn_pool.q.weight.copy_(_n2p(w[f'{mha_prefix}query/kernel'], t=False).flatten(1).T)
        module.attn_pool.q.bias.copy_(_n2p(w[f'{mha_prefix}query/bias'], t=False).reshape(-1))
        module.attn_pool.kv.weight.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/kernel'], t=False).flatten(1).T for n in ('key', 'value')]))
        module.attn_pool.kv.bias.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/bias'], t=False).reshape(-1) for n in ('key', 'value')]))
        module.attn_pool.proj.weight.copy_(_n2p(w[f'{mha_prefix}out/kernel']).flatten(1))
        module.attn_pool.proj.bias.copy_(_n2p(w[f'{mha_prefix}out/bias']))
        module.attn_pool.norm.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/scale']))
        module.attn_pool.norm.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/bias']))
        for r in range(2):
            getattr(module.attn_pool.mlp, f'fc{r + 1}').weight.copy_(_n2p(w[f'{block_prefix}MlpBlock_0/Dense_{r}/kernel']))
            getattr(module.attn_pool.mlp, f'fc{r + 1}').bias.copy_(_n2p(w[f'{block_prefix}MlpBlock_0/Dense_{r}/bias']))

open_clip.convert.load_big_vision_weights = load_any_bv_vit


def qknorm_update(open_clip_model,np_save_path):
    big_vision_vit = u.npload(np_save_path)
    # check if qknorm is present
    qknorm_present = any(['query_ln' in k for k in big_vision_vit.keys()])
    print(f"\nqknorm_present: {qknorm_present}")

    if qknorm_present:
        head_dim = open_clip_model.visual.trunk.blocks[0].attn.head_dim
        for i, blk in enumerate(open_clip_model.visual.trunk.blocks):
            # Replace query norm
            blk.attn.q_norm = torch.nn.LayerNorm(head_dim, eps=1e-6)
            blk.attn.q_norm.weight.data = torch.from_numpy(big_vision_vit[f'params/img/Transformer/encoderblock_{i}/MultiHeadDotProductAttention_0/query_ln/scale'])
            blk.attn.q_norm.bias.data = torch.zeros_like(blk.attn.q_norm.bias)  # Set bias to zero as it's not present in the checkpoint
            # Replace key norm
            blk.attn.k_norm = torch.nn.LayerNorm(head_dim, eps=1e-6)
            blk.attn.k_norm.weight.data = torch.from_numpy(big_vision_vit[f'params/img/Transformer/encoderblock_{i}/MultiHeadDotProductAttention_0/key_ln/scale'])
            blk.attn.k_norm.bias.data = torch.zeros_like(blk.attn.k_norm.bias)  # Set bias to zero as it's not present in the checkpoint
        print(f"Replaced query_ln and key_ln for {len(open_clip_model.visual.trunk.blocks)} blocks.")

        def qknorm_param_are_diff(threshold=1e-6):
            q_diffs,k_diffs = [],[]
            for i, blk in enumerate(open_clip_model.visual.trunk.blocks):
                oc_q = blk.attn.q_norm.weight.data.mean().item()
                oc_k = blk.attn.k_norm.weight.data.mean().item()
                bv_q = big_vision_vit[f'params/img/Transformer/encoderblock_{i}/MultiHeadDotProductAttention_0/query_ln/scale'].mean()
                bv_k = big_vision_vit[f'params/img/Transformer/encoderblock_{i}/MultiHeadDotProductAttention_0/key_ln/scale'].mean()
                q_diff = oc_q - bv_q
                k_diff = oc_k - bv_k
                q_diffs.append(q_diff)
                k_diffs.append(k_diff)
            q_mean_diff = np.mean(q_diffs)
            k_mean_diff = np.mean(k_diffs)
            print(f"q_norm diff mean: {q_mean_diff:.6f}, std: {np.std(q_diffs):.6f}")
            print(f"k_norm diff mean: {k_mean_diff:.6f}, std: {np.std(k_diffs):.6f}")    
            return abs(q_mean_diff) > threshold or abs(k_mean_diff) > threshold

        if qknorm_param_are_diff():
            print("query_ln and key_ln parameters are different.")
        else:
            print("query_ln and key_ln parameters are the same.")

    return open_clip_model

In [7]:
open_clip_model_type = "ViT-B-16-SigLIP" # whatever bv model you want to load, use SigLIP to call
model_kwargs = {
    'vision_cfg': {
    'image_size': 224, 
    'timm_model_name': 'vit_base_patch16_siglip_224', 
    'timm_model_pretrained': False, 
    'timm_pool': 'map', 
    'timm_proj': 'none'
  }, 
}
if backbone.split('_')[0] != 'siglip': model_kwargs['vision_cfg']['timm_pool'] = ''

open_clip_model, processor = open_clip.create_model_from_pretrained(
    open_clip_model_type,
    pretrained=local_np_save_path,
    # precision='bf16',
    **model_kwargs
)

open_clip_model = qknorm_update(open_clip_model,np_save_path=local_np_save_path)

open_clip_model

HELLO! I changed the load_checkpoint function!!!!

qknorm_present: False


CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768

# Output Consistency: big_vision vs. open_clip

In [8]:
dtype = 'float32' # float32, bfloat16
torch_dtype = torch.bfloat16 if dtype == 'bfloat16' else torch.float32
num_tests = 5

def torch2jnp(tensor,dtype):
    if dtype == 'bfloat16':
        temp = tensor.float().numpy()
        return jnp.array(temp, dtype='bfloat16')
    else:
        temp = tensor.numpy()
        return jnp.array(temp)

def cosine_similarity(a, b):
    if a.ndim == 2: 
        a = a[None]
        b = b[None]
    def sim(a_i, b_i):
        return np.dot(a_i, b_i) / (np.linalg.norm(a_i) * np.linalg.norm(b_i))
    sim_list = [sim(a_i, b_i) for a_i, b_i in zip(a[0], b[0])]
    return np.mean(sim_list)

similarity_list = []
for i in range(num_tests):
    print(f"test {i}")
    seed = i
    torch.manual_seed(seed)

    # initialize dummy image with same values
    open_clip_dummy_img = torch.randn(2, 3, 224, 224).to(torch_dtype)
    bv_dummy_img = torch2jnp(open_clip_dummy_img.permute(0, 2, 3, 1), dtype)
    print('dummy images initialized')

    # open_clip_dummy_txt = torch.randint(0, 32000, (2, 64)).to(torch.int64)
    # bv_dummy_txt = torch2jnp(open_clip_dummy_txt, 'int32')
    # print('dummy texts initialized')

    oc_out = open_clip_model(open_clip_dummy_img)[0]
    print('open_clip output generated')

    if backbone.split('_')[0] == 'cappa':
        bv_out = bv_model.apply({"params":params}, bv_dummy_img, return_enc_features=True, method = 'encode')[0]
    elif backbone.split('_')[0] == 'coca':
        bv_out = bv_model.apply({"params":params}, bv_dummy_img, return_enc_features=True, method = 'encode')[0]
    elif backbone.split('_')[0] == 'clip':
        bv_out = bv_model.apply({"params":params}, bv_dummy_img)[2]['img/encoded']
    elif 'gemma' in backbone:
        bv_out = bv_model.apply({"params":params}, bv_dummy_img, method='embed_image')[1]['img/pre_logits']
    else:
        bv_out = bv_model.apply({"params":params}, bv_dummy_img)[0]
    print('big_vision output generated')
    similarity = cosine_similarity(oc_out.float().detach().numpy(), bv_out)
    print('similarity:', similarity)
    similarity_list.append(similarity)
    sum(similarity_list) / num_tests
print(f"average similarity across {num_tests} tests: {sum(similarity_list) / num_tests}")

test 0
dummy images initialized
open_clip output generated
big_vision output generated
similarity: 0.99831533
test 1
dummy images initialized
open_clip output generated
big_vision output generated
similarity: 0.99868137
test 2
dummy images initialized
open_clip output generated
big_vision output generated
similarity: 0.9961379
test 3
dummy images initialized
open_clip output generated
big_vision output generated
similarity: 0.99832404
test 4
dummy images initialized
open_clip output generated
big_vision output generated
similarity: 0.9983085
average similarity across 5 tests: 0.9979534149169922


## similarity debug

In [9]:
# # cappa debugging code
# seed = 0
# torch.manual_seed(seed)
# dtype = 'bfloat16' # float32, bfloat16
# torch_dtype = torch.bfloat16 if dtype == 'bfloat16' else torch.float32

# def torch2jnp(tensor,dtype):
#     if dtype == 'bfloat16':
#         temp = tensor.float().numpy()
#         return jnp.array(temp, dtype='bfloat16')
#     else:
#         temp = tensor.numpy()
#         return jnp.array(temp)

# def cosine_similarity(a, b):
#     print(f"shapes: {a.shape}, {b.shape}")
#     print(f"shapes: {a.squeeze().shape}, {b.squeeze().shape}")
#     def sim(a_i, b_i):
#         return np.dot(a_i, b_i) / (np.linalg.norm(a_i) * np.linalg.norm(b_i))
#     sim_list = [sim(a_i, b_i) for a_i, b_i in zip(a.squeeze(), b.squeeze())]
#     return np.mean(sim_list)


# # initialize dummy image with same values
# open_clip_dummy_img = torch.randn(1, 3, 224, 224).to(torch_dtype)
# bv_dummy_img = torch2jnp(open_clip_dummy_img.permute(0, 2, 3, 1), dtype)
# print('dummy images initialized')

# oc_out = open_clip_model(open_clip_dummy_img)[0]
# bv_out = bv_model.apply({"params":params}, bv_dummy_img, return_enc_features=True, method = 'encode')
# print('out_similarity:', cosine_similarity(oc_out.float().detach().numpy(), bv_out[0]))

# oc_stem = open_clip_model.visual.trunk.patch_embed(open_clip_dummy_img)
# bv_stem = bv_out[1]['stem'].reshape(1,-1,768)
# stem_similarity = cosine_similarity(oc_stem.float().detach().numpy(), bv_stem)
# print(f"stem_similarity: {stem_similarity}")

# oc_pos = open_clip_model.visual.trunk._pos_embed(oc_stem)
# bv_pos = bv_out[1]['with_posemb']
# pos_similarity = cosine_similarity(oc_pos.float().detach().numpy(), bv_pos)
# print(f"pos_similarity: {pos_similarity}")

# oc_blocs = open_clip_model.visual.trunk.blocks(oc_pos)
# oc_encoded = open_clip_model.visual.trunk.norm(oc_blocs)
# bv_encoded = bv_out[1]['encoded']
# encoded_similarity = cosine_similarity(oc_encoded.float().detach().numpy(), bv_encoded)
# print(f"encoded_similarity: {encoded_similarity}")